In [317]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import itertools
import numpy as np
import random
import torch

In [2]:
url = "https://www.fantasynutmeg.com/api/history/season/2016-17"

In [3]:
data = requests.get(url)

In [4]:
hist = data.json()['history']

In [5]:
df = pd.DataFrame(columns=['Name','ID','Position'])

In [6]:
for i in range(len(hist)):
    name = hist[i]['first_name']+" "+hist[i]['second_name']
    id_ = hist[i]['id']
    pos = hist[i]['position']
    df.loc[i] = [name,id_,pos]

In [7]:
players = ["Alexis Sánchez","Gary Cahill","César Azpilicueta","Eden Hazard","Diego Da Silva Costa","Leighton Baines","Harry Kane","Romelu Lukaku","Sergio Agüero","Zlatan Ibrahimovic","Bamidele Alli","Christian Eriksen","Kevin De Bruyne","Kyle Walker","Marcos Alonso"]
df = df[df['Name'].isin(players)]

In [8]:
df

,Name,ID,Position
11,Alexis Sánchez,12,MID
76,César Azpilicueta,77,DEF
77,Gary Cahill,78,DEF
81,Eden Hazard,82,MID
96,Diego Da Silva Costa,97,FWD
124,Leighton Baines,125,DEF
142,Romelu Lukaku,143,FWD
234,Kevin De Bruyne,235,MID
238,Sergio Agüero,239,FWD
271,Zlatan Ibrahimovic,272,FWD


Top 5 players by position for the season(16-17)

Kane, Lukaku, Costa, Aguero, Ibrahimovic  
Sanchez, Alli, Hazard, Erikson, De Bruyne  
Cahill, Alonso, Azpi, Walker, Baines  
Heaton, Lloris, Courtois, de Gea, Cech

Sanchez: https://www.fantasynutmeg.com/api/history/season/2016-17?player=12  
403, 143, 97, 239, 272  
12, 398, 82, 394, 235  
78, 561, 77, 384, 125

In [9]:
player_stats = {}

In [10]:
data = requests.get("https://www.fantasynutmeg.com/api/history/season/2016-17?player=12")

In [11]:
data.json()["history"][0]

{'assists': 0,
 'attempted_passes': 17,
 'big_chances_created': 0,
 'big_chances_missed': 0,
 'bonus': 0,
 'bps': 0,
 'clean_sheets': 0,
 'clearances_blocks_interceptions': 0,
 'completed_passes': 12,
 'creativity': 15.3,
 'dribbles': 1,
 'ea_index': 0,
 'element': 12,
 'errors_leading_to_goal': 0,
 'errors_leading_to_goal_attempt': 0,
 'fixture': 8,
 'fouls': 0,
 'goals_conceded': 4,
 'goals_scored': 0,
 'ict_index': 3.5,
 'id': 12,
 'influence': 5.4,
 'key_passes': 1,
 'kickoff_time': '2016-08-14T15:00:00Z',
 'kickoff_time_formatted': '14 Aug 16:00',
 'loaned_in': 0,
 'loaned_out': 0,
 'minutes': 90,
 'offside': 4,
 'open_play_crosses': 0,
 'opponent_team': 9,
 'opponent_team_name': 'LIV',
 'opponent_team_promoted': False,
 'opponent_team_relegated': False,
 'own_goals': 0,
 'penalties_conceded': 0,
 'penalties_missed': 0,
 'penalties_saved': 0,
 'recoveries': 0,
 'red_cards': 0,
 'round': 1,
 'saves': 0,
 'selected': 296455,
 'tackled': 4,
 'tackles': 0,
 'target_missed': 0,
 'team_

In [12]:
for i in range(len(df)):
    temp_dict = {}
    data = requests.get("https://www.fantasynutmeg.com/api/history/season/2016-17?player="+str(df.iloc[i]["ID"]))
    data = data.json()["history"]
    print(len(data))
    for j,obj in enumerate(data):
        other_dict = {} 
        other_dict["cost"] = obj["value"]
        other_dict["points"] = obj["total_points"]
        temp_dict[obj["round"]] = other_dict
    
    player_stats[df.iloc[i]["Name"]] = temp_dict

38
38
38
38
38
38
38
38
38
38
38
38
38
38
35


In [13]:
player_stats["Marcos Alonso"]

{4: {'cost': 60, 'points': 0},
 5: {'cost': 60, 'points': 0},
 6: {'cost': 59, 'points': 1},
 7: {'cost': 59, 'points': 6},
 8: {'cost': 59, 'points': 6},
 9: {'cost': 59, 'points': 8},
 10: {'cost': 59, 'points': 6},
 11: {'cost': 59, 'points': 13},
 12: {'cost': 61, 'points': 6},
 13: {'cost': 61, 'points': 2},
 14: {'cost': 62, 'points': 6},
 15: {'cost': 63, 'points': 6},
 16: {'cost': 63, 'points': 7},
 17: {'cost': 64, 'points': 6},
 18: {'cost': 64, 'points': 6},
 19: {'cost': 64, 'points': 3},
 20: {'cost': 65, 'points': 1},
 21: {'cost': 65, 'points': 21},
 22: {'cost': 66, 'points': 6},
 23: {'cost': 66, 'points': 2},
 24: {'cost': 66, 'points': 8},
 25: {'cost': 67, 'points': 2},
 26: {'cost': 68, 'points': 2},
 27: {'cost': 68, 'points': 2},
 29: {'cost': 68, 'points': 6},
 30: {'cost': 68, 'points': 1},
 31: {'cost': 68, 'points': 2},
 32: {'cost': 68, 'points': 8},
 33: {'cost': 68, 'points': 0},
 34: {'cost': 68, 'points': 4},
 35: {'cost': 68, 'points': 6},
 36: {'cost'

In [14]:
attackers = []
midfielders = []
defenders = []

for i in range(len(df)):
    if df.iloc[i]["Position"] == "MID":
        midfielders.append(df.iloc[i]["Name"])
    elif df.iloc[i]["Position"] == "FWD":
        attackers.append(df.iloc[i]["Name"])
    else:
        defenders.append(df.iloc[i]["Name"])

In [15]:
attackers

['Diego Da Silva Costa',
 'Romelu Lukaku',
 'Sergio Agüero',
 'Zlatan Ibrahimovic',
 'Harry Kane']

In [16]:
midfielders

['Alexis Sánchez',
 'Eden Hazard',
 'Kevin De Bruyne',
 'Christian Eriksen',
 'Bamidele Alli']

In [17]:
defenders

['César Azpilicueta',
 'Gary Cahill',
 'Leighton Baines',
 'Kyle Walker',
 'Marcos Alonso']

In [167]:
a = [1,2,3,4,5]
m = [1,2,3,4,5]
d = [1,2,3,4,5]

In [168]:
state_space = []
for combination_a in itertools.combinations(a, 3):
    for combination_b in itertools.combinations(m, 3):
        for combination_c in itertools.combinations(d, 3):
            state_space.append({
                'attack': combination_a,
                'midfield': combination_b,
                'defence': combination_c
            })

In [169]:
len(state_space)

1000

In [170]:
state = state_space[420]
state

{'attack': (1, 3, 5), 'midfield': (1, 2, 5), 'defence': (1, 2, 3)}

In [22]:
week = 1

In [23]:
attack_ranking = []
midfield_ranking = []
defence_ranking = []

for i in attackers:
    temp_obj={}
    temp_obj["Name"] = i
    temp_obj["Points"] = player_stats[i][week]["points"]
    temp_obj["Cost"] = player_stats[i][week]["cost"]
    attack_ranking.append(temp_obj)

In [24]:
attack_ranking

[{'Name': 'Diego Da Silva Costa', 'Points': 5, 'Cost': 95},
 {'Name': 'Romelu Lukaku', 'Points': 0, 'Cost': 90},
 {'Name': 'Sergio Agüero', 'Points': 9, 'Cost': 130},
 {'Name': 'Zlatan Ibrahimovic', 'Points': 9, 'Cost': 115},
 {'Name': 'Harry Kane', 'Points': 2, 'Cost': 110}]

In [25]:
attack_ranking = sorted(attack_ranking, key = lambda x : -x["Points"])

In [26]:
attack_ranking

[{'Name': 'Sergio Agüero', 'Points': 9, 'Cost': 130},
 {'Name': 'Zlatan Ibrahimovic', 'Points': 9, 'Cost': 115},
 {'Name': 'Diego Da Silva Costa', 'Points': 5, 'Cost': 95},
 {'Name': 'Harry Kane', 'Points': 2, 'Cost': 110},
 {'Name': 'Romelu Lukaku', 'Points': 0, 'Cost': 90}]

In [27]:
for i in state["attack"]:
    print(i)

1
3
5


In [234]:
def get_state_players(week,state):
    attack_ranking = []
    midfield_ranking = []
    defence_ranking = []

    for i in attackers:
        temp_obj={}
        temp_obj["Name"] = i
        if week in player_stats[i].keys():
            temp_obj["Points"] = player_stats[i][week]["points"]
            temp_obj["Cost"] = player_stats[i][week]["cost"]
        else:
            temp_obj["Points"] = -100
            temp_obj["Cost"] = 200
        attack_ranking.append(temp_obj)
    for i in midfielders:
        temp_obj={}
        temp_obj["Name"] = i
        if week in player_stats[i].keys():
            temp_obj["Points"] = player_stats[i][week]["points"]
            temp_obj["Cost"] = player_stats[i][week]["cost"]
        else:
            temp_obj["Points"] = -100
            temp_obj["Cost"] = 200
        midfield_ranking.append(temp_obj)
    for i in defenders:
        temp_obj={}
        temp_obj["Name"] = i
        if week in player_stats[i].keys():
            temp_obj["Points"] = player_stats[i][week]["points"]
            temp_obj["Cost"] = player_stats[i][week]["cost"]
        else:
            temp_obj["Points"] = -100
            temp_obj["Cost"] = 200
        defence_ranking.append(temp_obj)
        
    attack_ranking = sorted(attack_ranking, key = lambda x : -x["Points"])
    midfield_ranking = sorted(midfield_ranking, key = lambda x : -x["Points"])
    defence_ranking = sorted(defence_ranking, key = lambda x : -x["Points"])
    
    return_obj = {}
    return_att = []
    return_mid = []
    return_def = []
    reward = 0
    cost = 0
    
    for i in state["attack"]:
        return_att.append(attack_ranking[i-1])
        reward+= attack_ranking[i-1]["Points"]
        cost+= attack_ranking[i-1]["Cost"]
        
    for i in state["midfield"]:
        return_mid.append(midfield_ranking[i-1])
        reward+= midfield_ranking[i-1]["Points"]
        cost+= midfield_ranking[i-1]["Cost"]
        
    for i in state["defence"]:
        return_def.append(defence_ranking[i-1])
        reward+= defence_ranking[i-1]["Points"]
        cost+= defence_ranking[i-1]["Cost"]
    
    if cost > 900:
        reward -=10

    return_obj["attack"] = return_att
    return_obj["midfield"] = return_mid   
    return_obj["defence"] = return_def
        
    next_att = []
    next_mid = []
    next_def = []
    
    for i in attackers:
        temp_obj={}
        temp_obj["Name"] = i
        if week+1 in player_stats[i].keys():
            temp_obj["Points"] = player_stats[i][week+1]["points"]
            temp_obj["Cost"] = player_stats[i][week+1]["cost"]
        else:
            temp_obj["Points"] = -100
            temp_obj["Cost"] = 200
        next_att.append(temp_obj)
    for i in midfielders:
        temp_obj={}
        temp_obj["Name"] = i
        if week+1 in player_stats[i].keys():
            temp_obj["Points"] = player_stats[i][week+1]["points"]
            temp_obj["Cost"] = player_stats[i][week+1]["cost"]
        else:
            temp_obj["Points"] = -100
            temp_obj["Cost"] = 200
        next_mid.append(temp_obj)
    for i in defenders:
        temp_obj={}
        temp_obj["Name"] = i
        if week+1 in player_stats[i].keys():
            temp_obj["Points"] = player_stats[i][week+1]["points"]
            temp_obj["Cost"] = player_stats[i][week+1]["cost"]
        else:
            temp_obj["Points"] = -100
            temp_obj["Cost"] = 200
        next_def.append(temp_obj)
    
    next_att = sorted(next_att, key = lambda x : -x["Points"])
    next_mid = sorted(next_mid, key = lambda x : -x["Points"]) 
    next_def = sorted(next_def, key = lambda x : -x["Points"])
    
    att_state = []
    mid_state = []
    def_state = []
    
    for obj in return_att:
        att_state.append(next((i+1 for i, item in enumerate(next_att) if item["Name"] == obj["Name"]), None))
    
    for obj in return_mid:
        mid_state.append(next((i+1 for i, item in enumerate(next_mid) if item["Name"] == obj["Name"]), None))
    
    for obj in return_def:
        def_state.append(next((i+1 for i, item in enumerate(next_def) if item["Name"] == obj["Name"]), None))
    
    new_state = {
        'attack': tuple(sorted(att_state)),
        'midfield': tuple(sorted(mid_state)),
        'defence': tuple(sorted(def_state))
    }
    
    return reward,return_obj,new_state

In [212]:
state

{'attack': (1, 3, 5), 'midfield': (1, 2, 5), 'defence': (1, 2, 3)}

In [226]:
get_state_players(38,state)

(65,
 {'attack': [{'Name': 'Harry Kane', 'Points': 17, 'Cost': 119},
   {'Name': 'Romelu Lukaku', 'Points': 8, 'Cost': 101},
   {'Name': 'Zlatan Ibrahimovic', 'Points': 0, 'Cost': 114}],
  'midfield': [{'Name': 'Bamidele Alli', 'Points': 11, 'Cost': 91},
   {'Name': 'Alexis Sánchez', 'Points': 9, 'Cost': 117},
   {'Name': 'Christian Eriksen', 'Points': 9, 'Cost': 89}],
  'defense': [{'Name': 'Gary Cahill', 'Points': 6, 'Cost': 67},
   {'Name': 'Leighton Baines', 'Points': 3, 'Cost': 58},
   {'Name': 'César Azpilicueta', 'Points': 2, 'Cost': 64}]},
 {'attack': (2, 4, 5), 'midfield': (1, 4, 5), 'defence': (2, 3, 1)})

In [215]:
new_state

{'attack': (1, 3, 5), 'midfield': (1, 3, 5), 'defence': (1, 2, 4)}

In [36]:
state_space.index(new_state)

441

In [38]:
action_space = [0,1,2,3,4,5,6]

In [98]:
def generate_binary_numbers_with_3_ones():
    numbers = []
    for i in range(32):  # 32 represents all possible 5-digit binary numbers
        binary = bin(i)[2:].zfill(5)  # Convert to binary and pad with zeros to ensure 5 digits
        if binary.count('1') == 3:  # Check if the number of ones is exactly 3
            numbers.append(binary)
    return numbers

# Generate and print all 5-digit binary numbers with 3 ones
binary_numbers = generate_binary_numbers_with_3_ones()

def get_bin(tuple_numbers):
    binary_number = ['0'] * 5  # Initialize a list to represent the binary number

    for index in tuple_numbers:
        binary_number[index - 1] = '1'  # Set the bit at the specified index to 1

    return ''.join(binary_number)  # Convert the list to a string

def get_tup(binary_number):
    tuple_numbers = tuple(index + 1 for index, bit in enumerate(binary_number) if bit == '1')
    return tuple_numbers

In [189]:
def take_action(state,action):
    
    new_state = state.copy()
    
    if action == 1:
        index = binary_numbers.index(get_bin(new_state["attack"]))
        index = max(0,index-1)
        new_state["attack"] = tuple(sorted(get_tup(binary_numbers[index])))
    elif action == 2:
        index = binary_numbers.index(get_bin(new_state["attack"]))
        index = min(9,index+1)
        new_state["attack"] = tuple(sorted(get_tup(binary_numbers[index])))
    elif action == 3:
        index = binary_numbers.index(get_bin(new_state["midfield"]))
        index = max(0,index-1)
        new_state["midfield"] = tuple(sorted(get_tup(binary_numbers[index])))
    elif action == 4:
        index = binary_numbers.index(get_bin(new_state["midfield"]))
        index = min(9,index+1)
        new_state["midfield"] = tuple(sorted(get_tup(binary_numbers[index])))
    elif action == 5:
        index = binary_numbers.index(get_bin(new_state["defence"]))
        index = max(0,index-1)
        new_state["defence"] = tuple(sorted(get_tup(binary_numbers[index])))
    elif action == 6:
        index = binary_numbers.index(get_bin(new_state["defence"]))
        index = min(9,index+1)
        new_state["defence"] = tuple(sorted(get_tup(binary_numbers[index])))
    return new_state

In [264]:
gamma = 0.9
threshold = 0.01

In [246]:
def value_iteration(gamma,threshold):
    V = np.zeros(len(state_space),dtype=np.float32)
    iteration = 1
    while True:
        V_temp= np.empty(len(state_space),dtype=np.float32)
        for state in state_space:
            V_actions = np.zeros(len(action_space))
            for action in action_space:
                for week in range(1,39):
                    temp_state = take_action(state,action)
                    reward,obj,next_state = get_state_players(week,temp_state)
                    index = state_space.index(next_state)
                    V_actions[action] += reward + gamma * V[index]
                V_actions[action] /=38
            cur_index = state_space.index(state)
            V_temp[cur_index] = np.max(V_actions)
        max_delta = np.max(np.abs(V - V_temp))
        print(iteration,max_delta)
        iteration +=1
        V = V_temp.copy()
        if max_delta <= threshold:
            break
    
    return V

In [250]:
V = value_iteration(gamma,threshold)

1 45.894737
2 12.490997
3 5.2615166
4 2.5348434
5 1.2572441
6 0.62685585
7 0.3131504
8 0.15653038
9 0.07826233


In [268]:
def extract_optimal_policy(V_optimal, gamma):
    optimal_policy = torch.zeros(len(state_space))
    for state in state_space:
        v_actions = torch.zeros(len(action_space))
        for action in action_space:
            for week in range(1,39):
                temp_state = take_action(state,action)
                reward,obj,next_state = get_state_players(week,temp_state)
                index = state_space.index(next_state)
                v_actions[action] += reward + gamma * V_optimal[index]
            v_actions[action] /=38
        cur_index = state_space.index(state)
        optimal_policy[cur_index] = torch.argmax(v_actions)
    return optimal_policy

In [272]:
optimal_policy_val_iter = extract_optimal_policy(V, gamma)

In [290]:
optimal_policy_val_iter

tensor([0., 6., 6., 0., 6., 5., 0., 6., 6., 6., 4., 4., 6., 4., 6., 5., 4., 6.,
        4., 4., 4., 4., 6., 4., 6., 5., 4., 6., 4., 4., 0., 6., 6., 0., 6., 5.,
        0., 6., 6., 6., 4., 4., 6., 4., 6., 5., 4., 6., 4., 4., 4., 4., 6., 4.,
        6., 5., 4., 6., 4., 4., 0., 6., 6., 0., 6., 5., 0., 6., 6., 6., 4., 4.,
        6., 4., 6., 5., 4., 6., 4., 4., 4., 4., 6., 4., 6., 5., 4., 6., 4., 4.,
        4., 4., 6., 4., 6., 5., 4., 6., 4., 4., 2., 2., 6., 2., 6., 5., 2., 6.,
        2., 2., 2., 2., 6., 2., 6., 5., 2., 6., 2., 2., 2., 2., 6., 2., 6., 5.,
        2., 6., 2., 2., 2., 2., 6., 2., 6., 5., 2., 6., 2., 2., 2., 2., 6., 2.,
        6., 5., 2., 6., 2., 2., 2., 2., 6., 2., 6., 5., 2., 6., 2., 2., 2., 2.,
        6., 2., 6., 5., 2., 6., 2., 2., 2., 2., 6., 2., 6., 5., 2., 6., 2., 2.,
        2., 2., 6., 2., 6., 5., 2., 6., 2., 2., 2., 2., 6., 2., 6., 5., 2., 6.,
        2., 2., 2., 2., 6., 2., 6., 5., 2., 6., 2., 2., 2., 2., 6., 2., 6., 5.,
        2., 6., 2., 2., 2., 2., 6., 2., 

In [386]:
from collections import defaultdict

def gen_epsilon_greedy_policy(n_action, epsilon):
    def policy_function(state,Q):
        index = state_space.index(state)
        probs = torch.ones(n_action) * epsilon / n_action
        best_action = torch.argmax(Q[index]).item()
        probs[best_action] += 1.0 - epsilon
        action = torch.multinomial(probs,1).item()
        return action
    return policy_function

def q_learning(gamma,n_episode,alpha):
    Q = defaultdict(lambda: torch.zeros(len(action_space)))
    for episode in range(n_episode):
        state_random_index = random.randint(0,999)
        state = state_space[state_random_index]
        for week in range(1,39):
            # check from here
            action = epsilon_greedy_policy(state,Q)
            temp_state = take_action(state,action)
            index_state = state_space.index(state)
            reward, obj, next_state = get_state_players(week,temp_state)
            index_next = state_space.index(next_state)
            td_delta = reward + gamma * torch.max(Q[index_next]) - Q[index_state][action]
            Q[index_state][action] += alpha * td_delta
            
            state = next_state
    policy = {}
    for state, actions in Q.items():
        policy[state] = torch.argmax(actions).item()
    return Q, policy

In [399]:
gamma = 0.9
n_episode = 10000

alpha = 0.4
epsilon = 0.2

epsilon_greedy_policy = gen_epsilon_greedy_policy(len(action_space), epsilon)

optimal_Q, optimal_policy = q_learning(gamma, n_episode, alpha)

In [388]:
len(optimal_Q)

1000

In [389]:
1000 in optimal_policy.keys()

False

In [400]:
num = len(optimal_Q)

optimal_policy_Q_learning = torch.zeros(num)
for key in optimal_policy.keys():
    optimal_policy_Q_learning[key] = optimal_policy[key]

In [300]:
optimal_policy_Q_learning

tensor([2., 1., 5., 4., 0., 1., 1., 0., 0., 2., 6., 1., 4., 1., 6., 2., 2., 3.,
        5., 1., 2., 3., 1., 4., 0., 6., 1., 6., 0., 3., 0., 0., 0., 3., 1., 2.,
        0., 0., 1., 0., 0., 1., 0., 1., 0., 6., 0., 0., 0., 3., 0., 1., 1., 4.,
        0., 6., 1., 0., 2., 2., 0., 3., 1., 3., 3., 0., 0., 2., 2., 4., 0., 0.,
        0., 0., 3., 4., 1., 0., 0., 3., 2., 0., 0., 4., 0., 5., 1., 0., 5., 0.,
        0., 0., 6., 1., 0., 0., 6., 1., 3., 1., 4., 0., 1., 0., 5., 4., 2., 0.,
        0., 1., 6., 4., 2., 2., 5., 6., 3., 1., 1., 1., 5., 2., 6., 0., 0., 0.,
        0., 1., 0., 1., 1., 4., 0., 3., 2., 2., 0., 0., 0., 4., 4., 1., 4., 1.,
        0., 3., 4., 2., 3., 0., 1., 2., 0., 6., 0., 6., 0., 0., 5., 2., 3., 0.,
        0., 2., 0., 4., 0., 0., 1., 0., 4., 0., 0., 2., 0., 1., 0., 0., 0., 5.,
        6., 4., 0., 6., 1., 0., 3., 0., 0., 0., 0., 1., 2., 0., 1., 1., 3., 1.,
        2., 4., 0., 6., 5., 2., 1., 0., 5., 0., 0., 0., 1., 0., 3., 3., 0., 3.,
        2., 3., 2., 0., 4., 0., 0., 0., 

In [379]:
array1 = optimal_policy_val_iter.numpy()
array2 = optimal_policy_Q_learning.numpy()

# Find the number of different values
num_different_values = np.sum(array1 != array2)

print("Number of different values:", num_different_values)

Number of different values: 840


In [401]:
state = state_space[69]

reward_Val_Iter = 0
for week in range(1,39):
    index = state_space.index(state)
    action = int(optimal_policy_val_iter[index])
    temp_state = take_action(state,action)
    reward, obj, next_state = get_state_players(week,temp_state)
    reward_Val_Iter += reward
    state = next_state

In [402]:
print(reward_Val_Iter)

727


In [413]:
state = state_space[1]

reward_Q_learning = 0
for week in range(1,39):
    index = state_space.index(state)
    action = int(optimal_policy_Q_learning[index])
    temp_state = take_action(state,action)
    reward, obj, next_state = get_state_players(week,temp_state)
    reward_Q_learning += reward
    state = next_state

In [414]:
print(reward_Q_learning)

763


In [411]:
state = state_space[1]


reward_Random = 0
for iter_ in range(10):
    for week in range(1,39):
        index = state_space.index(state)
        action = random.randint(0,6)
        temp_state = take_action(state,action)
        reward, obj, next_state = get_state_players(week,temp_state)
        reward_Random += reward
        state = next_state
reward_Random /= 10

In [412]:
print(reward_Random)

454.4
